In this project we will be analyzing the conent of Hackernews posts, and analyze their popularity. 

We will be using the following dataset:
https://www.kaggle.com/hacker-news/hacker-news-posts

It is a filtered dataset that takes a random sample of the submissions to Hackernews that have any activity on the submission (i.e they have a caomment), and then tandomly sampled. 

The headers are as follows:

id: The unique identifier from Hacker News for the post
title: The title of the post
url: The URL that the posts links to, if it the post has a URL
num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
num_comments: The number of comments that were made on the post
author: The username of the person who submitted the post
created_at: The date and time at which the post was submitted



In [1]:
from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

print(hn[:5])


[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


We Will now clear up the headers:


In [2]:
headers = hn[0]
hn = hn[1:]

print(headers)

print('\n')
#printing rows to verify

print(hn[:4])


['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


We will now be looking to filter by the category of posts which have ask HN or show HN , to investigate how well these do on average. These are 2 types of community posts, so we will investiagte if community posts tend to do better than other news articles and submissions. 

Luckily, when it is one of those types of posts, Hackernews starts the tile of the questions with Ask HN, and so we can simply extrapolate the entires which start with that particular string of 'Ask HN'.

To do this we will create a loop to extrapolate a list with the requied titlkes. 


In [3]:
# initiate empty lists to operate on in the loop for the output

ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn') == True:
        ask_posts.append(row)
    elif title.lower().startswith('show hn') == True:
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print('ask_posts length is ' + str(len(ask_posts)))        
print('show_posts length is ' + str(len(show_posts)))        
print('other_posts length is ' + str(len(other_posts)))

print('proportion of community posts is ' + str((len(ask_posts) + len(show_posts)) / len(hn)))



ask_posts length is 1744
show_posts length is 1162
other_posts length is 17194
proportion of community posts is 0.1445771144278607


Now that we have isolated the types of posts we will see check their engagement on the comment basis:

'num_comments' is column with index 4

Firstly we will investigate the 'ask' posts.


In [4]:
# set a ask comments list, to track total number of asks.
# then find avg comments on an ask post

total_ask_comments = 0

for row in ask_posts:
    num_comments = row[4]
    num_comments = int(num_comments)
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments / len(ask_posts)

print('avg ask comments: ' + str(avg_ask_comments))

# set a ask comments list, to track total number of asks.
# then find avg comments on an ask post

total_show_comments = 0

for row in show_posts:
    num_comments = row[4]
    num_comments = int(num_comments)
    total_show_comments += num_comments

avg_show_comments = total_show_comments / len(show_posts)

print('avg show comments: ' + str(avg_show_comments))


avg ask comments: 14.038417431192661
avg show comments: 10.31669535283993


From what we can see above, the average Ask comments receive around 14 comments on avareage, vs show comments of 10.3. 

This is somewhat expects, as the ask HN posts, directly ask for community engagement, and such prompting more comments to be created. 



Based on this we wil focus our analysis of the posts of these type, as they seem to acheive good community engagement. 

We will now try to see if the popularity of a post, is related to the time posted. 

To do this we will calculate:
1. Amount of ask posts created in each hour of the dat, alongside the number of comments received
2. Average number of comments the ask posts receive by the hour created

We also note that the the time in the database is stored as a string, so we will want to convert these in to objects, so taht we can manipulate them more easily. 

For Reference, a sample element looks like the following:

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']





In [5]:
from datetime import datetime

result_list = []



for row in ask_posts:
    created_at = row[6]
    comments = row[4]
    result_list.append([created_at, comments])

# Test: check if loop produces desired data     
print(result_list[:5])

#We will now intialize dicitionaries, to track the number of posts in an hour
#We will also count comments in a specific hour. 
# we will then use the two to analyze Comments per posts in a specific hour to gage popularity

counts_by_hour = {}
comments_by_hour = {}

#note : the used implementation of datetime does not require differentaion between padded and unpadded numbers

for row in result_list:
    hour = row[0]
    comments = int(row[1])
    hour_object = datetime.strptime(hour, '%m/%d/%Y %H:%M')
    target_hour = hour_object.hour
    if target_hour not in counts_by_hour:
        counts_by_hour[target_hour] = 1
        comments_by_hour[target_hour] = comments
    else:
        counts_by_hour[target_hour] += 1
        comments_by_hour[target_hour] += comments

print('\n')        
print("These are the post counts by hour:\n")
print(counts_by_hour)
print('\n')   
print("These are the comment counts by hour: \n")
print(comments_by_hour)



[['8/16/2016 9:55', '6'], ['11/22/2015 13:43', '29'], ['5/2/2016 10:14', '1'], ['8/2/2016 14:20', '3'], ['10/15/2015 16:38', '17']]


These are the post counts by hour:

{0: 55, 1: 60, 2: 58, 3: 54, 4: 47, 5: 46, 6: 44, 7: 34, 8: 48, 9: 45, 10: 59, 11: 58, 12: 73, 13: 85, 14: 107, 15: 116, 16: 108, 17: 100, 18: 109, 19: 110, 20: 80, 21: 109, 22: 71, 23: 68}


These are the comment counts by hour: 

{0: 447, 1: 683, 2: 1381, 3: 421, 4: 337, 5: 464, 6: 397, 7: 267, 8: 492, 9: 251, 10: 793, 11: 641, 12: 687, 13: 1253, 14: 1416, 15: 4477, 16: 1814, 17: 1146, 18: 1439, 19: 1188, 20: 1722, 21: 1745, 22: 479, 23: 543}


We have now succesfully populated 2 dictionaries:

*counts_by_hour* : contains the number of ask posts created during each hour of the day.

*comments_by_hour* : contains the corresponding number of comments ask posts created at each hour received.

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

To do so, we will create a list of lists, where the first entry is the hour and the second element is the average number of comments per post, in that hour.

In [6]:
# first we create a list based on the dictionaries so that we can perform opertaions

avg_by_hour = []


for row in counts_by_hour:
    avg_by_hour.append([row,comments_by_hour[row] / counts_by_hour[row]])

print('\n')
print('The following list is the average comments per post , grouped by hour posted'+ '\n')
print(*avg_by_hour, sep = "\n" )





The following list is the average comments per post , grouped by hour posted

[0, 8.127272727272727]
[1, 11.383333333333333]
[2, 23.810344827586206]
[3, 7.796296296296297]
[4, 7.170212765957447]
[5, 10.08695652173913]
[6, 9.022727272727273]
[7, 7.852941176470588]
[8, 10.25]
[9, 5.5777777777777775]
[10, 13.440677966101696]
[11, 11.051724137931034]
[12, 9.41095890410959]
[13, 14.741176470588234]
[14, 13.233644859813085]
[15, 38.5948275862069]
[16, 16.796296296296298]
[17, 11.46]
[18, 13.20183486238532]
[19, 10.8]
[20, 21.525]
[21, 16.009174311926607]
[22, 6.746478873239437]
[23, 7.985294117647059]


We will now order the Produced list, to identify the best posting time. Listed printed above, can sometimes print unordered (although the python collision resolution wil usually print it in correct order, given that we have provided well-orderd inputs).

We will then isolate the top 5 best posting hours, to summarize the data.


In [7]:
# These 2 lines will sort the avg_by_hour list by the avg 
# post numer, reverse sort ( i.e descending) it and print it


avg_by_hour.sort(key = lambda avg_by_hour: avg_by_hour[1], reverse = True) 

sorted_swap = avg_by_hour


print(*sorted_swap, sep = "\n" )



[15, 38.5948275862069]
[2, 23.810344827586206]
[20, 21.525]
[16, 16.796296296296298]
[21, 16.009174311926607]
[13, 14.741176470588234]
[10, 13.440677966101696]
[14, 13.233644859813085]
[18, 13.20183486238532]
[17, 11.46]
[1, 11.383333333333333]
[11, 11.051724137931034]
[19, 10.8]
[8, 10.25]
[5, 10.08695652173913]
[12, 9.41095890410959]
[6, 9.022727272727273]
[0, 8.127272727272727]
[23, 7.985294117647059]
[7, 7.852941176470588]
[3, 7.796296296296297]
[4, 7.170212765957447]
[22, 6.746478873239437]
[9, 5.5777777777777775]


We will now clean up the presentation and display the top 5 candidate posting hours:

In [8]:
print("Top 5 Hours to post for 'Ask' Posts based on Average Comments")
print('\n')

for row in sorted_swap[:4]:
    output = "{}:00 : {:.2f} average comments per post".format(str(row[0]), row[1])
    print(output)

Top 5 Hours to post for 'Ask' Posts based on Average Comments


15:00 : 38.59 average comments per post
2:00 : 23.81 average comments per post
20:00 : 21.52 average comments per post
16:00 : 16.80 average comments per post


We Also note that this is based on US Eastern time. Based on the data documentation. This also the project author's (Matt's) time zone.

We can see that there are no morning hours for the post times, and the most populaar post hour is 15:00. This might be due to the fact, that the portal itself will have low readership during work hours and earlier.

Posts posted after 15:00 will have the benefit of gaining traffic and being visibility once everyone finishes work in the east coast. Once traction has been gained, traffic could inflow from the west coast in the evening as well - so the effects would accumulate. 

An interesting finding is that posts sent at 2 in the morning are quite popular - this would require deeper investigation. It could be that, these posts would be ready for the early morning readers, and upvoted by any readers that have picked it up at night. 

That being said, the analysis of these posts is based on averages for times, and comments. There other factors to gage post popularity such as social media shares, and the quality of the article itself. The effect of this time is indicative of a correlation - not neccessarily a strong causal link.

However, this top 5 could give an inidication of when to post an 'Ask' article, to get the most traction on average. 






